In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
dir = "/content/drive/My Drive/NLP_CA4_Question2_Data"

train = pd.read_excel(dir+"/train.xlsx")
test = pd.read_excel(dir+"/test.xlsx")
valid = pd.read_excel(dir+"/valid.xlsx")
del dir

Mounted at /content/drive


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 51.0 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
encoded_dict = {'bible':0, 'quran':1, 'mizan':2}
train['category'] = train.category.map(encoded_dict)
test['category'] = test.category.map(encoded_dict)
valid['category'] = valid.category.map(encoded_dict)

In [4]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train.category)
y_test = to_categorical(test.category)
y_valid = to_categorical(valid.category)

In [5]:
import transformers

In [6]:
from transformers import RobertaTokenizer, RobertaModel, TFXLMRobertaModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = TFXLMRobertaModel.from_pretrained("jplu/tf-xlm-roberta-base")

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76G [00:00<?, ?B/s]

Some layers from the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [7]:
x_train = tokenizer(
    text=train.source.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test.targets.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_valid = tokenizer(
    text=valid.source.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [8]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [9]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Input, Dense

In [10]:
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = model(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [11]:
optimizer = Adam(
    learning_rate=3e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
    )

loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [12]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = ({'input_ids':x_valid['input_ids'],'attention_mask':x_valid['attention_mask']}, y_valid),
    epochs=10,
    batch_size=32
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


394/394 [==============================] - 392s 946ms/step - loss: 0.2122 - balanced_accuracy: 0.9195 - val_loss: 0.0980 - val_balanced_accuracy: 0.9689
Epoch 2/10
394/394 [==============================] - 374s 948ms/step - loss: 0.0684 - balanced_accuracy: 0.9780 - val_loss: 0.0799 - val_balanced_accuracy: 0.9770
Epoch 3/10
394/394 [==============================] - 374s 948ms/step - loss: 0.0479 - balanced_accuracy: 0.9840 - val_loss: 0.0737 - val_balanced_accuracy: 0.9789
Epoch 4/10
394/394 [==============================] - 374s 949ms/step - loss: 0.0358 - balanced_accuracy: 0.9888 - val_loss: 0.0745 - val_balanced_accuracy: 0.9785
Epoch 5/10
394/394 [==============================] - 374s 949ms/step - loss: 0.0263 - balanced_accuracy: 0.9913 - val_loss: 0.0765 - val_balanced_accuracy: 0.9793
Epoch 6/10
394/394 [==============================] - 374s 949ms/step - loss: 0.0230 - balanced_accuracy: 0.9927 - val_loss: 0.0695 - val_balanced_accuracy: 0.9793
Epoch 7/10
394/394 [=======

In [13]:
import numpy as np
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = test.category
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.92      0.53      0.67       900
           1       0.73      0.87      0.79       900
           2       0.79      0.98      0.88       900

    accuracy                           0.79      2700
   macro avg       0.81      0.79      0.78      2700
weighted avg       0.81      0.79      0.78      2700

